<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# [AutoKeras](https://autokeras.com/)

** Note: This code has been tested as of July 2021**

As AI is automating more and more jobs, it can finally automate designing AI architectures too. Neural Architecture Search (NAS) approaches utilize reinforcement learning to join together mini architectural blocks, till they are able to maximize the objective function - i.e. our validation accuracy. The current state of the art networks are all based on NAS, leaving human-designed architectures in the dust. Research in this area started showing promising results in 2017, with a bigger focus on making train faster in 2018.

AutoKeras (Haifeng Jin et al), also apply this state of the art technique on our particular datasets in a relatively accessible manner. Generating new model architectures with AutoKeras is a matter of supplying our images and associated labels, and a time limit to finish running the jobs by. Internally, it implements several optimization algorithms including a Bayesian optimization approach to search for an optimal architecture.

In [ ]:
!pip install tensorflow-gpu==2.5.0
!pip3 install autokeras
!pip install keras-tuner --upgrade

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak

Load the standard MNIST data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Define an image classifier. The `max_trials` setting can be played around with to enable more experimentation. It refers to the maximum number of different Keras Models to try. The search may finish before reaching the max_trials. Defaults to 100.

In [6]:
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=10)

Fit the classifier.

In [7]:
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 01m 57s]
val_loss: 0.06357701867818832

Best val_loss So Far: 0.06357701867818832
Total elapsed time: 00h 01m 57s
INFO:tensorflow:Oracle triggered exit
1875/1875 [==============================] - 138s 73ms/step - loss: 0.1578 - accuracy: 0.9520
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


Prediction based on the best model trained

In [8]:
predicted_y = clf.predict(x_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_true=y_test, y_pred=predicted_y))

313/313 [==============================] - 5s 16ms/step
0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:191: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


Evaluate the best model with testing data

In [9]:
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 5s 17ms/step - loss: 0.0589 - accuracy: 0.9799
[0.058876268565654755, 0.9799000024795532]


Export as a Keras Model

In [10]:
model = clf.export_model()

Let's visualize the trained model.

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28)            0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0     

In [12]:
print(type(model))

<class 'tensorflow.python.keras.engine.functional.Functional'>


Now, let's write the model to disk

In [13]:
try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

INFO:tensorflow:Assets written to: model_autokeras/assets


Let's read the model we just wrote.

In [14]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))